In [1]:
import os
import requests
from dotenv import load_dotenv
import clickhouse_connect
import itertools
import pandas as pd
import json
load_dotenv()
clickhouse_host = os.getenv("CLICKHOUSE_HOST")
username = os.getenv("CLICKHOUSE_USER")
password = os.getenv("CLICKHOUSE_PASSWORD")
github_token = os.getenv("GITHUB_TOKEN")

client = clickhouse_connect.get_client(host=clickhouse_host, port=8123, username=username, password=password)

headers = {
  "Authorization": f"token {github_token}"
}

/Users/xiaoyawork/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 绘制领域 OpenRank 变化曲线

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline

# AI Coding
data_lists = [
    [191, 391],
    [19, 102, 195],
    [109, 133, 147, 163, 184, 175, 181],
    [71, 93, 98, 110, 118, 126, 139],
    [143, 166, 175, 179, 180, 146, 134],
    [120, 127, 118, 131, 139, 135, 132],
    [62, 65, 73, 83, 85, 86, 82],
    [137, 130, 96, 81],
    [57, 75, 89, 79, 81, 79, 60],
    [116, 124, 120, 104, 87, 68, 57],
    [128, 128, 116, 110, 99, 73, 50],
    [23, 28, 28, 28, 28, 26, 22],
    [233, 219, 214, 194, 178, 82, 22],
    [39, 42, 40, 39, 37, 21, 13]
]


# 计算每月总和
month_sums = np.zeros(7, dtype=int)
for lst in data_lists:
    L = len(lst)
    start = 7 - L
    for i, v in enumerate(lst):
        month_sums[start + i] += v

months = [f"2025-{i:02d}" for i in range(1, 8)]

print(month_sums)

x = np.arange(7)

# 平滑曲线
xs = np.linspace(x.min(), x.max(), 300)
spline = make_interp_spline(x, month_sums, k=3)
ys = spline(xs)

fig, ax = plt.subplots(figsize=(9, 5))

# 光晕描边
ax.plot(xs, ys, color='deeppink', linewidth=10, alpha=0.1, solid_capstyle='round')
ax.plot(xs, ys, color='deeppink', linewidth=4, alpha=0.6, solid_capstyle='round')
ax.plot(xs, ys, color='deeppink', linewidth=2.5, solid_capstyle='round')

for i, val in enumerate(month_sums):
    ax.text(x[i], val+12, str(val), ha='center', va='bottom',
            fontsize=8, fontweight='bold', color='grey')
  
# 设置横纵坐标
ax.set_xticks(x)
ax.set_xticklabels(months, fontsize=13, fontweight='bold')   # 横轴刻度加粗放大
ax.set_xlabel("Month", fontsize=12, fontweight='bold')       # 横轴名称加粗放大

ax.set_ylabel("OpenRank", fontsize=14, fontweight='bold')    # 纵轴名称加粗放大
for label in ax.get_yticklabels():
  label.set_fontsize(13)
  label.set_fontweight('bold')
ax.tick_params(axis='y', labelsize=13, width=0.5)            # 纵轴刻度加粗放大

# 只保留横向网格线
ax.yaxis.grid(True, linestyle='--', alpha=0.5)
ax.xaxis.grid(False)


# 去掉多余的边框
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(True)

plt.title("AI Coding", fontsize=16, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()


## 绘制单项目 OpenRank 变化曲线

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import make_interp_spline

# 数据
all_months = ['1月', '2月', '3月', '4月', '5月', '6月', '7月']
values = [26, 47, 79, 98, 115, 125, 135] # mastra
title = " "  # 配置图片标题

values = [15, 15, 15,15,15, 191, 391] #gemini-cli


# 根据 values 的长度动态调整 months，从 7 月往前递减
months = all_months[-len(values):]

# 创建 x 轴数据点
x = np.arange(len(values))

# 检查数据点数量，如果少于4个点，使用线性插值或直接绘制
if len(values) < 4:
    # 对于少于4个数据点，使用线性插值或直接连线
    xs = np.linspace(x.min(), x.max(), 300)
    ys = np.interp(xs, x, values)
else:
    # 对于4个或更多数据点，使用三次样条插值，但设置边界条件避免端点异常波动
    xs = np.linspace(x.min(), x.max(), 300)
    # 使用 bc_type='natural' 来设置自然边界条件，减少端点的异常波动
    # 或者使用 bc_type='clamped' 并设置端点导数为0
    spline = make_interp_spline(x, values, k=3, bc_type='natural')
    ys = spline(xs)
    
    # 另一种方法：限制插值结果不超出原始数据的范围
    # 这可以防止样条插值在端点产生过度的振荡
    min_val = min(values)
    max_val = max(values)
    ys = np.clip(ys, min_val, max_val)

fig, ax = plt.subplots(figsize=(8, 5))

# 光晕描边效果 - 黑粉色
ax.plot(xs, ys, color='deeppink', linewidth=10, alpha=0.1, solid_capstyle='round')
ax.plot(xs, ys, color='deeppink', linewidth=6, alpha=0.3, solid_capstyle='round')
ax.plot(xs, ys, color='deeppink', linewidth=3, alpha=0.8, solid_capstyle='round')

# # 添加原始数据点
# ax.scatter(x, values, color='white', s=40, zorder=3, alpha=0.6, edgecolors='white', linewidth=2)

# # 在数据点上方标注数值
# for i, val in enumerate(values):
#     ax.text(x[i], val + max(values) * 0.05, str(val), ha='center', va='bottom',
#             fontsize=10, fontweight='bold', color='black')

# # 设置纵坐标范围
ax.set_ylim(0, 600)

# 隐藏所有坐标轴、刻度和网格
ax.set_xticks([])
ax.set_yticks([])
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.grid(False)

plt.title(title, fontsize=16, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()


## 绘制 OpenRank 对比曲线

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# ========== 原始 OpenRank 月度数据 ==========
pytorch = {
  "2017-01": 48.28, "2017-02": 78.11, "2017-03": 102.47, "2017-04": 124.25, "2017-05": 140.59, "2017-06": 158.01,
  "2017-07": 158.47, "2017-08": 164.62, "2017-09": 146.32, "2017-10": 183.08, "2017-11": 214.95, "2017-12": 228.92,
  "2018-01": 229.68, "2018-02": 235.58, "2018-03": 260.36, "2018-04": 319.6,  "2018-05": 346.6,  "2018-06": 373.28,
  "2018-07": 386.91, "2018-08": 386.34, "2018-09": 380.92, "2018-10": 424.16, "2018-11": 449.49, "2018-12": 447.57,
  "2019-01": 454.52, "2019-02": 453.15, "2019-03": 442.42, "2019-04": 484.79, "2019-05": 529.17, "2019-06": 538.43,
  "2019-07": 553.48, "2019-08": 606.99, "2019-09": 611.75, "2019-10": 645.01, "2019-11": 669.38, "2019-12": 666.98,
  "2020-01": 627.53, "2020-02": 607.91, "2020-03": 691.64, "2020-04": 736.29, "2020-05": 731.66, "2020-06": 728.53,
  "2020-07": 749.57, "2020-08": 761.96, "2020-09": 778.01, "2020-10": 784.24, "2020-11": 793.71, "2020-12": 783.52,
  "2021-01": 772.13, "2021-02": 731.59, "2021-03": 775.17, "2021-04": 801.53, "2021-05": 793.39, "2021-06": 798.42,
  "2021-07": 813.31, "2021-08": 822.49, "2021-09": 768.9,  "2021-10": 735.0,  "2021-11": 683.51, "2021-12": 689.45,
  "2022-01": 691.41, "2022-02": 672.15, "2022-03": 662.01, "2022-04": 666.36, "2022-05": 675.95, "2022-06": 706.81,
  "2022-07": 714.21, "2022-08": 730.75, "2022-09": 677.97, "2022-10": 715.07, "2022-11": 748.65, "2022-12": 755.35,
  "2023-01": 763.57, "2023-02": 775.87, "2023-03": 839.92, "2023-04": 869.99, "2023-05": 867.76, "2023-06": 863.55,
  "2023-07": 860.22, "2023-08": 894.71, "2023-09": 892.66, "2023-10": 882.84, "2023-11": 915.72, "2023-12": 861.46,
  "2024-01": 876.86, "2024-02": 897.87, "2024-03": 893.14, "2024-04": 928.08, "2024-05": 924.4,  "2024-06": 956.31,
  "2024-07": 951.57, "2024-08": 987.24, "2024-09": 961.54, "2024-10": 972.9,  "2024-11": 987.71, "2024-12": 965.42,
  "2025-01": 950.52, "2025-02": 928.65, "2025-03": 904.38, "2025-04": 924.69, "2025-05": 909.48, "2025-06": 907.32,
  "2025-07": 858.55, "2025-08": 834.83
}

tensorflow = {
  "2015-11": 104.81, "2015-12": 126.84, "2016-01": 148.15, "2016-02": 179.37, "2016-03": 179.82, "2016-04": 228.7,
  "2016-05": 250.7,  "2016-06": 295.57, "2016-07": 319.28, "2016-08": 331.5,  "2016-09": 355.82, "2016-10": 362.43,
  "2016-11": 396.57, "2016-12": 392.31, "2017-01": 380.64, "2017-02": 436.64, "2017-03": 491.23, "2017-04": 485.25,
  "2017-05": 521.01, "2017-06": 523.43, "2017-07": 507.45, "2017-08": 520.31, "2017-09": 474.51, "2017-10": 489.48,
  "2017-11": 522.36, "2017-12": 544.66, "2018-01": 579.74, "2018-02": 532.0,  "2018-03": 548.05, "2018-04": 556.7,
  "2018-05": 527.08, "2018-06": 516.24, "2018-07": 535.61, "2018-08": 536.25, "2018-09": 544.56, "2018-10": 564.45,
  "2018-11": 569.04, "2018-12": 543.07, "2019-01": 514.71, "2019-02": 561.45, "2019-03": 588.73, "2019-04": 636.57,
  "2019-05": 634.16, "2019-06": 660.46, "2019-07": 675.09, "2019-08": 737.33, "2019-09": 725.2,  "2019-10": 742.94,
  "2019-11": 751.13, "2019-12": 732.69, "2020-01": 666.82, "2020-02": 633.56, "2020-03": 696.5,  "2020-04": 726.98,
  "2020-05": 728.15, "2020-06": 739.11, "2020-07": 729.17, "2020-08": 712.37, "2020-09": 709.67, "2020-10": 669.28,
  "2020-11": 615.35, "2020-12": 567.35, "2021-01": 537.23, "2021-02": 501.98, "2021-03": 512.51, "2021-04": 493.37,
  "2021-05": 497.68, "2021-06": 472.07, "2021-07": 410.87, "2021-08": 376.49, "2021-09": 340.44, "2021-10": 311.0,
  "2021-11": 272.41, "2021-12": 267.37, "2022-01": 259.29, "2022-02": 229.88, "2022-03": 243.77, "2022-04": 233.59,
  "2022-05": 228.63, "2022-06": 230.48, "2022-07": 225.08, "2022-08": 247.42, "2022-09": 242.04, "2022-10": 247.28,
  "2022-11": 229.05, "2022-12": 231.78, "2023-01": 217.9,  "2023-02": 208.62, "2023-03": 210.59, "2023-04": 202.17,
  "2023-05": 191.01, "2023-06": 187.96, "2023-07": 179.57, "2023-08": 176.89, "2023-09": 163.88, "2023-10": 151.95,
  "2023-11": 142.89, "2023-12": 126.73, "2024-01": 120.19, "2024-02": 119.63, "2024-03": 127.17, "2024-04": 127.67,
  "2024-05": 116.45, "2024-06": 111.91, "2024-07": 116.91, "2024-08": 105.93, "2024-09": 99.96,  "2024-10": 98.11,
  "2024-11": 105.68, "2024-12": 104.4,  "2025-01": 94.62,  "2025-02": 87.44,  "2025-03": 71.13,  "2025-04": 59.31,
  "2025-05": 61.45,  "2025-06": 38.41,  "2025-07": 39.47,  "2025-08": 40.19
}

# ========== 构造 2015-11 到 2025-08 的“每 6 个月”采样时间轴 ==========
def month_iter(start_ym="2015-11", end_ym="2025-08", step_months=6):
    y, m = map(int, start_ym.split("-"))
    ye, me = map(int, end_ym.split("-"))
    cur = datetime(y, m, 1)
    end = datetime(ye, me, 1)
    out = []
    while cur <= end:
        out.append(f"{cur.year:04d}-{cur.month:02d}")
        # 前进 step_months 个月
        ny = cur.year + (cur.month - 1 + step_months) // 12
        nm = (cur.month - 1 + step_months) % 12 + 1
        cur = datetime(ny, nm, 1)
    return out

dates = month_iter("2015-09", "2025-09", 6)

# ========== 将两项目的数据映射到统一时间轴，并用 NaN 占位 ==========
def series_on_axis(data_dict, axis_labels):
    return [data_dict.get(ts, np.nan) for ts in axis_labels]

tf_series = series_on_axis(tensorflow, dates)
pt_series = series_on_axis(pytorch,   dates)  # 2015-11~2016-12 将自动是 NaN

# ========== 平滑（仅对各自“非 NaN”连续片段做样条）==========
from scipy.interpolate import make_interp_spline

def smooth_segments(y_values):
    """
    对含 NaN 的序列做分段样条：返回一组 (x_new, y_new) 片段，可被逐段绘制为一条平滑曲线。
    x 采用统一下标 0..N-1。
    """
    x = np.arange(len(y_values))
    segments = []
    start = None
    for i, v in enumerate(y_values + [np.nan]):  # 末尾加 NaN 触发收尾
        if not np.isnan(v) and start is None:
            start = i
        if (np.isnan(v) or i == len(y_values)) and start is not None:
            end = i  # [start, end)
            xs = x[start:end]
            ys = np.array(y_values[start:end], dtype=float)
            if len(xs) >= 3:
                # 样条：k=3；采样更密一点
                sp = make_interp_spline(xs, ys, k=3)
                xs_new = np.linspace(xs[0], xs[-1], max(200, 30*len(xs)))
                ys_new = sp(xs_new)
                segments.append((xs_new, ys_new))
            elif len(xs) == 2:
                # 两点就直线
                xs_new = np.linspace(xs[0], xs[-1], 200)
                ys_new = np.interp(xs_new, xs, ys)
                segments.append((xs_new, ys_new))
            elif len(xs) == 1:
                segments.append((xs, ys))
            start = None
    return segments

pt_segments = smooth_segments(pt_series)
tf_segments = smooth_segments(tf_series)

# ========== 绘图（16:9，坐标加粗，无网格，线条光晕）==========
plt.figure(figsize=(12, 9))

# 颜色
pt_color = "#FF1493"  # deeppink
tf_color = "#D8A7F2"  # light purple

def draw_glow_line(segments, color, lw=3):
    # 先画发光“外圈”几层
    for alpha, grow in [(0.12, 12), (0.10, 9), (0.08, 6)]:
        for xs, ys in segments:
            plt.plot(xs, ys, color=color, linewidth=lw+grow, alpha=alpha, solid_joinstyle="round", solid_capstyle="round")
    # 再画主体线
    for xs, ys in segments:
        plt.plot(xs, ys, color=color, linewidth=lw, solid_joinstyle="round", solid_capstyle="round")

draw_glow_line(tf_segments, tf_color, lw=3)
draw_glow_line(pt_segments, pt_color, lw=3)

# 散点（可选，弱化一点作为锚点）
x_idx = np.arange(len(dates))
plt.scatter(x_idx, tf_series, s=18, color=tf_color, alpha=0.35)
plt.scatter(x_idx, pt_series, s=18, color=pt_color, alpha=0.35)

# 坐标轴 & 刻度
plt.xticks(x_idx, dates, rotation=35, ha="right", fontsize=13, weight="bold")
plt.yticks(fontsize=13, weight="bold")
plt.xlabel(" ", fontsize=16, weight="bold")
plt.ylabel("OpenRank", fontsize=16, weight="bold")
plt.title("  ", fontsize=18, weight="bold")

# 去网格
plt.grid(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# 图例：无边框、加粗，显示正确的颜色
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], color=tf_color, lw=3, label='TensorFlow'),
    Line2D([0], [0], color=pt_color, lw=3, label='PyTorch')
]
plt.legend(
    handles=legend_elements,
    loc="upper left",
    frameon=False,
    prop={"size": 16, "weight": "bold"},
    labelspacing=0.8, handlelength=2.2
)

plt.tight_layout()
plt.show()

## 根据 Landscape 中所有项目的 description，获得词云图

In [ ]:
# 将所有描述文本合并成一个字符串
text = """"""


# 定义常用词列表
common_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 
                'of', 'with', 'by', 'from', 'up', 'about', 'into', 'over', 'after',
                'is', 'are', 'was', 'were', 'be', 'been', 'being',
                'i', 'you', 'he', 'she', 'it', 'we', 'they',
                'this', 'that', 'these', 'those',
                'your', 'my', 'his', 'her', 'its', 'our', 'their'}

# 创建一个字典来存储单词频率
word_freq = {}

# 第一步：按空格分割
words = []
for word in text.lower().split():
    # 处理破折号分隔的单词
    if '-' in word:
        words.extend(word.split('-'))
    else:
        words.append(word)

# 第二步：按逗号分割
comma_words = []
for word in words:
    if ',' in word:
        comma_words.extend(word.split(','))
    else:
        comma_words.append(word)

# 更新words列表
words = comma_words

for word in words:
    # 去除标点符号
    word = word.strip('.,!?()[]{}":;-/&')
    # 确保单词不是空字符串且不是常用词
    if word and word not in common_words:
        # 将复数形式转为单数形式(简单的s结尾情况)
        if word.endswith('s') and word[:-1] in word_freq:
            base_word = word[:-1]
            word_freq[base_word] = word_freq.get(base_word, 0) + 1
        else:
            word_freq[word] = word_freq.get(word, 0) + 1

print(words)

# 按频率降序排序
sorted_word_freq = dict(sorted(word_freq.items(), key=lambda x: x[1], reverse=True))

print("去除常用词后的词频统计结果（前100个）：")
for word, freq in list(sorted_word_freq.items())[:100]:
    print(f"{word}: {freq}")

In [ ]:
# -*- coding: utf-8 -*-
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np

# ===== 词频数据（原样粘贴即可） =====
word_freq = {
    "AI": 126, "LLM": 95, "Agent": 80, "Data": 79, "Learning": 44, "Model": 38, "Search": 36, "OpenAI": 35,
    "Framework": 32, "Python": 30, "Workflow": 30, "MCP": 29, "Open": 28, "Source": 26, "DeepSeek": 24,
    "Vector": 23, "Inference": 23, "Language": 22, "Machine": 22, "RAG": 20, "Code": 19, "Deep": 19,
    "Agentic": 18, "Platform": 18, "Engine": 17, "Tool": 17, "Serving": 16, "Database": 15, "Large": 15,
    "Llama": 15, "Gemini": 13, "Application": 13, "Building": 12, "Multi": 12, "Chatbot": 12, "ChatGPT": 12,
    "PyTorch": 12, "Automation": 12, "GPU": 12, "Annotation": 12, "Evaluation": 12, "Engineering": 11,
    "Server": 11, "GPT": 11, "Ollama": 11, "Observability": 11, "Distributed": 11, "MLOps": 11, "Image": 11,
    "Use": 10, "LangChain": 10, "API": 10, "R1": 10, "Science": 10, "CUDA": 10, "System": 10, "Prompt": 10,
    "Intelligence": 9, "Memory": 9, "Self": 9, "Integration": 9, "Qwen3": 9, "Developer": 9, "Metadata": 9,
    "Anthropic": 8, "Artificial": 8, "Generative": 8, "Management": 8, "Cloud": 8, "Hosted": 8, "Document": 8,
    "Qwen": 8, "Pipeline": 8, "Apache": 8, "Labeling": 8, "Speech": 8, "Build": 7, "Tuning": 7, "LLMOps": 7,
    "Development": 7, "TypeScript": 7, "Library": 7, "Any": 7, "Claude": 7, "Llama3": 7, "Processing": 7,
    "Transformer": 7, "Performance": 7, "Spark": 7, "Neural": 7, "Fine": 6, "Multimodal": 6, "NextJS": 6,
    "UI": 6, "Production": 6, "No": 6, "Orchestration": 6, "Native": 6, "Retrieval": 6, "Knowledge": 6
}

# ===== 画幅：16:9 白底 =====
W, H = 1920, 1080
BG_COLOR = "#ffffff"

# ===== 生成“不规则椭圆”遮罩（白色=允许放词；黑色=禁止）=====
yy, xx = np.ogrid[:H, :W]
cx, cy = W / 2, H / 2

# 基础椭圆半径（控制留白程度；数值越小留白越多）
rx_base, ry_base = int(W * 0.38), int(H * 0.30)

# 角度场（相对中心）
theta = np.arctan2(yy - cy, xx - cx)  # [-pi, pi]

# 构造不规则半径：在椭圆基础上加入多频正弦扰动（幅度可调）
amp = 0.10   # 扰动幅度（0.05~0.15 之间比较自然）
perturb = (
    0.50*np.sin(1*theta) +
    0.35*np.sin(2*theta + 0.8) +
    0.15*np.sin(3*theta + 1.7)
) / (0.50+0.35+0.15)  # 归一，范围[-1,1]
rx = rx_base * (1 + amp * perturb)
ry = ry_base * (1 + amp * perturb)

# 椭圆方程（使用位置相关的 rx/ry，得到不规则边界）
mask_inside = ((xx - cx)**2) / (rx**2) + ((yy - cy)**2) / (ry**2) <= 1.0

# WordCloud 的 mask：白=255可放词，黑=0禁放
mask_img = np.where(mask_inside, 0, 255).astype(np.uint8)

# ===== 柔和 Aurora 渐变（粉→紫→蓝），高频更偏深一些以增强对比 =====
def hex_to_rgb(h): 
    h = h.lstrip('#'); return tuple(int(h[i:i+2], 16) for i in (0, 2, 4))
def rgb_to_hex(t): 
    return '#%02x%02x%02x' % t
def lerp(c1, c2, t): 
    return tuple(int(c1[i] + (c2[i]-c1[i]) * t) for i in range(3))
palette = ["#ffb3c1", "#ff85a1", "#d084e8", "#8fb4ff", "#4fa3ff"]
def build_gradient(palette, steps=256):
    cols = [hex_to_rgb(c) for c in palette]
    out, seg = [], len(cols) - 1
    for i in range(steps):
        t = i / (steps - 1)
        k = min(int(t * seg), seg - 1)
        local_t = (t * seg) - k
        out.append(rgb_to_hex(lerp(cols[k], cols[k+1], local_t)))
    return out
gradient = build_gradient(palette, steps=256)

# ===== 词云参数：强制横向；不占满画幅（由 mask 控制）；16:9 输出 =====
FONT_PATH = "RacingSansOne-Regular.ttf"  # <- 改为你的字体路径
wc = WordCloud(
    font_path=FONT_PATH,
    width=W, height=H,
    background_color=BG_COLOR,
    mask=mask_img,
    prefer_horizontal=1.0,     # 100% 横向
    relative_scaling=0.45,
    max_words=400,
    min_font_size=12,
    collocations=False,
    margin=2,
    scale=2                    # 提升清晰度
)

wc.generate_from_frequencies(word_freq)
freq_map = wc.words_  # 0~1 相对频率

# 高频更靠中心：WordCloud 内部会优先放大高频，结合椭圆区域自然更往中间
def color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    f = freq_map.get(word, 0.0)
    idx = int((0.35 + 0.65 * f) * (len(gradient) - 1))  # 避免太浅
    idx = max(0, min(len(gradient)-1, idx))
    return gradient[idx]

wc = wc.recolor(color_func=color_func, random_state=42)

# ===== 显示与保存 =====
plt.figure(figsize=(W/100, H/100), facecolor=BG_COLOR)
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

wc.to_file("wordcloud_irregular_ellipse_white_16x9.png")
print("Saved to: wordcloud_irregular_ellipse_white_16x9.png")
